# [![](https://img.shields.io/badge/Thiết%20kế-stablediffusion.vn-0075ff)](https://stablediffusion.vn) [![](https://img.shields.io/badge/Phiên%20bản-v1.5-0075ff)](https://stablediffusion.vn) [![](https://img.shields.io/badge/Group-Support-0075ff)](https://www.facebook.com/groups/stablediffusion.vn) [![](https://img.shields.io/badge/Bộ%20công%20cụ-Đầy%20đủ-0075ff)](https://stablediffusion.vn/bo-cong-cu/) [![](https://img.shields.io/discord/813085864355037235?color=blue&label=Discord&logo=Discord)](https://discord.gg/5SEtApPeyG) [![](https://img.shields.io/badge/Donate-SDVN-green)](https://stablediffusion.vn/donate/) 

---
# 💡 Hướng dẫn sử dụng :
 
 
 - B1 : Chuẩn bị thư mục ảnh mẫu để train theo đúng kích thước (ex:512x512 , 768x768), tải lên drive
 - B1 : Chạy phần 1
 - B2 : Chỉnh Path để dẫn đến thư mục data đã tải và chạy phần 2
 - B3 : Chỉnh các thông số train tại mục 3 và chạy phần 3
 - B4 : Chạy mục 4 và theo dõi kết quả train
 ---

 🔻 : Lưu ý quan trọng thay đổi theo mỗi lần train

 🔸 : Giá trị tham khảo, có thể để mặc định

 🔹 : Giá trị nên để mặc định

 *Các giá trị mặc định được tối ưu cho ảnh train mẫu người với 10 ảnh, các thư mục mặc định được tối ưu cho drive sử  dụng SD-WebUI trên stablediffusion.vn*


# 🔌 1. Cài đặt nền

In [ ]:
#@title

#Dreamboot Lora by stablediffusion.vn - hungdiffusion - 24/03/2023
#Lora Dreamboot origin - kohya

import os
import zipfile
import shutil
from subprocess import getoutput
from IPython.utils import capture
from google.colab import drive
%store -r

!nvidia-smi
!pip install -q torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1+cu118 torchtext==0.15.1 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!pip install -q xformers==0.0.19 triton==2.0.0 -U


#root_dir
root_dir = "/content"
deps_dir = os.path.join(root_dir,"deps")
repo_dir = os.path.join(root_dir,"kohya-trainer")
training_dir = os.path.join(root_dir,"LoRA")
pretrained_model = os.path.join(root_dir,"pretrained_model")
vae_dir = os.path.join(root_dir,"vae")
config_dir = os.path.join(training_dir,"config")

#repo_dir
accelerate_config = os.path.join(repo_dir, "accelerate_config/config.yaml")
tools_dir = os.path.join(repo_dir,"tools")
finetune_dir = os.path.join(repo_dir,"finetune")

for store in ["root_dir", "deps_dir", "repo_dir", "training_dir", "pretrained_model", "vae_dir", "accelerate_config", "tools_dir", "finetune_dir", "config_dir"]:
  with capture.capture_output() as cap:
    %store {store}
    del cap

repo_url = "https://github.com/phamhungd/kohya-trainer"
branch = "" 
install_xformers = True 
mount_drive = True

if mount_drive:
  if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

for dir in [deps_dir, training_dir, config_dir, pretrained_model, vae_dir]:
  os.makedirs(dir, exist_ok=True)
  
def clone_repo(url):
  if not os.path.exists(repo_dir):
    os.chdir(root_dir)
    !git clone {url} {repo_dir}
  else:
    os.chdir(repo_dir)
    !git pull origin {branch} if branch else !git pull

clone_repo(repo_url)

if branch:
  os.chdir(repo_dir)
  status = os.system(f"git checkout {branch}")
  if status != 0:
    raise Exception("Failed to checkout branch or commit")

os.chdir(repo_dir)

def ubuntu_deps(url, name, dst):
  with capture.capture_output() as cap:
    !wget -q --show-progress {url}
    with zipfile.ZipFile(name, 'r') as deps:
      deps.extractall(dst)
    !dpkg -i {dst}/*
    os.remove(name)
    shutil.rmtree(dst)
    del cap 

def install_dependencies():
  !pip -q install --upgrade -r requirements.txt

  if install_xformers:
    !pip install xformers triton

  from accelerate.utils import write_basic_config
  if not os.path.exists(accelerate_config):
    write_basic_config(save_location=accelerate_config)

os.chdir(repo_dir)
ubuntu_deps("https://huggingface.co/Linaqruf/fast-repo/resolve/main/deb-libs.zip", "deb-libs.zip", deps_dir)
install_dependencies()

# 📂 2. Kết nối data Train

In [ ]:

#@markdown ##<br>🔻 2.1 Đường dẫn thư mục dữ liệu trong drive 
#@markdown <br>
import os
from IPython.utils import capture
import random
import concurrent.futures
from tqdm import tqdm
from PIL import Image
%store -r


Train_Data_Path = "/content/drive/MyDrive/SD-Data/TrainData/" #@param {type:'string'}
#@markdown <br> ✅  --- *`Tệp hỗ trợ : ".png", ".jpg", ".jpeg", ".webp", ".bmp"`*
#@markdown <br> ❌  --- *`Tệp không hỗ trợ : ".PNG", ".JPG", ".JPEG", ".WEBP", ".BMP"`*
train_data_dir = Train_Data_Path
reg_data_dir = train_data_dir

for image_dir in [train_data_dir, reg_data_dir]:
  if image_dir:
    with capture.capture_output() as cap:
      os.makedirs(image_dir, exist_ok=True)
      %store image_dir
      del cap

print(f"Your train data directory : {train_data_dir}")

os.chdir(root_dir)

test = os.listdir(train_data_dir)
os.chdir(finetune_dir)
#@markdown <br> Bỏ chọn delete_metadata nếu không muốn tự tạo tag ở mục 2.2 :
#@markdown <br>
delete_metadata = False # @param {type: "boolean"}

if not delete_metadata:
  supported_types = [".png", ".jpg", ".jpeg", ".PNG", ".JPG", ".JPEG", ".webp", ".bmp", ".caption", ".npz", ".txt", ".json"]
else:
  supported_types = [".png", ".jpg", ".jpeg", ".PNG", ".JPG", ".JPEG", ".webp", ".bmp"]

for item in test:
    file_ext = os.path.splitext(item)[1]
    if file_ext not in supported_types:
        print(f"Deleting file {item} from {train_data_dir}")
        os.remove(os.path.join(train_data_dir, item))

#@markdown ##<br>🔸 2.2 Tạo tag mô tả cho ảnh (Waifu Diffusion 1.4 Tagger V2) :
#@markdown <br> Chỉnh threshold thấp (0.35) đối với train style, khung cảnh , chỉnh threshold cao (0.85) đối với train nhân vật, đối tượng, chỉnh 1 để không tự tạo tag :
#@markdown <br>

batch_size = 8 
max_data_loader_n_workers = 2 
model = "SmilingWolf/wd-v1-4-swinv2-tagger-v2"
threshold = 0.85 #@param {type:"slider", min:0, max:1, step:0.05}
if delete_metadata == True :
 !python tag_images_by_wd14_tagger.py \
  "{train_data_dir}" \
  --batch_size {batch_size} \
  --repo_id {model} \
  --thresh {threshold} \
  --caption_extension .txt \
  --max_data_loader_n_workers {max_data_loader_n_workers}

#@markdown Nối thêm tag tuỳ chọn vào đầu txt :
extension = ".txt"
custom_tag = "" #@param {type:"string"}
#@markdown Tích chọn append để customtag xuất hiện ở cuối mô tả :
append = False #@param {type:"boolean"}
if extension != "" :
 def add_tag(filename, tag, append):
    with open(filename, "r") as f:
        contents = f.read()
        	    	
    tag = ", ".join(tag.split())
    tag = tag.replace("_", " ")
    
    if tag in contents:
        return

    if append:
      contents = contents.rstrip() + ", " + tag
    else:
      contents = tag + ", " + contents
    
    with open(filename, "w") as f:
        f.write(contents)

 if not any([filename.endswith("." + extension) for filename in os.listdir(train_data_dir)]):
    for filename in os.listdir(train_data_dir):
        if filename.endswith((".png", ".jpg", ".jpeg", ".webp", ".bmp")):
            open(os.path.join(train_data_dir, filename.split(".")[0] + "." + extension), "w").close()

 tags = custom_tag.split()

 if custom_tag:
  for filename in os.listdir(train_data_dir):
      if filename.endswith(extension):
          for tag in tags:
              add_tag(os.path.join(train_data_dir, filename), tag, append)

# ⚙️ 3. Cài đặt Train

In [ ]:
from google.colab import drive
#@markdown ##<br>🔻 3.1 Cài đặt Lora :
#@markdown <br>
v_parameterization = False
Lora_name = "" #@param {type:"string"}
project_name = Lora_name
if not project_name:
  project_name = "Loratrain"
#@markdown <br>*Để trống model train để kích hoạt custom model* 
#ModelTrain
modelDown = ["",
             "aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/cag/anything-v3-1/resolve/main/anything-v3-1.safetensors -d /content/model -o AnythingV3.safetensors",
             "aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Linaqruf/stolen/resolve/main/pruned-models/chillout_mix-pruned.safetensors -d /content/model -o Chilloutmix.safetensors",
             "aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Linaqruf/stolen/resolve/main/pruned-models/stable_diffusion_1_5-pruned.safetensors -d /content/model -o StableDiffusionV15.safetensors",
             "aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.safetensors -d /content/model -o v2-StableDiffusionV21-512.safetensors",
             "aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors -d /content/model -o v2-StableDiffusionV21-768.safetensors",
             "aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/wd-1-4-anime_e2.ckpt -d /content/model -o v2-WaijuDiffusionV14.safetensors",]

vaeDown = ["",
           "aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/phamhungd/GuoZovya/resolve/main/Anime.ckpt -d /content/VAE -o Animevae.vae.pt",
           "aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime.ckpt -d /content/VAE -o Kl-f8-anime.vae.pt",
           "aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d /content/VAE -o VAE84EMA.vae.pt",]
modelPath = ["",
            "/content/model/AnythingV3.safetensors",
            "/content/model/Chilloutmix.safetensors",
            "/content/model/StableDiffusionV15.safetensors",
            "/content/model/v2-StableDiffusionV21-512.safetensors",
            "/content/model/v2-StableDiffusionV21-768.safetensors",
            "/content/model/v2-WaijuDiffusionV14.safetensors",]
modelList = ["",
             "AnythingV3",
             "Chilloutmix",
             "StableDiffusionV15",
             "v2-StableDiffusionV21-512",
             "v2-StableDiffusionV21-768",
             "v2-WaijuDiffusionV14",]

Model_Train_Name = "Chilloutmix"  #@param ["", "AnythingV3", "Chilloutmix", "StableDiffusionV15", "v2-StableDiffusionV21-512", "v2-StableDiffusionV21-768", "v2-WaijuDiffusionV14"]
Custom_Model_Path = "" #@param {type:"string"}
v2 = False #@param {type:"boolean"}

if Model_Train_Name != "":
  !{modelDown[modelList.index(Model_Train_Name)]}
  pretrained_model_name_or_path=modelPath[modelList.index(Model_Train_Name)]
else:
  pretrained_model_name_or_path=Custom_Model_Path
#@markdown <br>*Để trống VAE để không kích hoạt VAE* 
vaePath = [ "",
            "/content/VAE/Animevae.vae.pt",
            "/content/VAE/Kl-f8-anime.vae.pt",
            "/content/VAE/VAE84EMA.vae.pt",]
vaeList = ["",
           "Animevae",
           "Kl-f8-anime",
           "VAE84EMA",]
VAE = "VAE84EMA"  #@param ["", "Animevae", "Kl-f8-anime", "VAE84EMA"]
!{vaeDown[vaeList.index(VAE)]}
vae = vaePath[vaeList.index(VAE)]
Output_Path = "/content/drive/MyDrive/SD-Data/Lora/" #@param {'type':'string'}
output_dir = Output_Path
output_to_drive = False
print("Project Name: ", project_name)
print("Model Version: Stable Diffusion V1.x") if not v2 else ""
print("Model Version: Stable Diffusion V2.x") if v2 and not v_parameterization else ""
print("Model Version: Stable Diffusion V2.x 768v") if v2 and v_parameterization else ""
print("Pretrained Model Path: ", pretrained_model_name_or_path) if pretrained_model_name_or_path else print("No Pretrained Model path specified.")
print("VAE Path: ", vae) if vae else print("No VAE path specified.")
print("Output Path: ", output_dir)

import toml
#@markdown ##<br>🔻 3.2 Cấu hình Dreambooth :
#@markdown ##<br>
#@markdown  *Số lượt lặp lại ảnh trong thư mục :*
train_repeats = 48 #@param {type:"number"}
reg_repeats = 1 
instance_token = "mksks" 
class_token = "style"
#@markdown  *Kích thước ảnh train (px) :*
resolution = 512 # @param {type:"slider", min:512, max:1024, step:128}
#@markdown <br>*Cài đặt số ảnh xử lý trong 1 step:*
#@markdown <br>```resolution = 512 => b= 6```
#@markdown <br>```resolution = 768 => b= 4```
#@markdown <br>```resolution = 896 => b= 2```
#@markdown <br>```resolution = 1024 => b= 1```
Batch_size = 6 #@param {type:"number"}
flip_aug = False 
caption_extension = ".txt" 
caption_dropout_rate = 0 
caption_dropout_every_n_epochs = 0 
keep_tokens = 0 

config = {
    "general": {
        "enable_bucket": True,
        "caption_extension": caption_extension,
        "shuffle_caption": True,
        "keep_tokens": keep_tokens,
        "bucket_reso_steps": 64,
        "bucket_no_upscale": False,
    },
    "datasets": [
        {
            "resolution": resolution,
            "min_bucket_reso": 320 if resolution > 640 else 256,
            "max_bucket_reso": 1280 if resolution > 640 else 1024,           
            "caption_dropout_rate": caption_dropout_rate if caption_extension == ".caption" else 0,
            "caption_tag_dropout_rate": caption_dropout_rate if caption_extension == ".txt" else 0,
            "caption_dropout_every_n_epochs": caption_dropout_every_n_epochs,
            "flip_aug": flip_aug,
            "color_aug": False,
            "face_crop_aug_range": None,
            "subsets": [
                {
                    "image_dir": train_data_dir,
                    "class_tokens": f"{instance_token} {class_token}",
                    "num_repeats": train_repeats,
                },
                {
                    "is_reg": True,
                    "image_dir": reg_data_dir,
                    "class_tokens": class_token,
                    "num_repeats": reg_repeats,
                }
            ]
        }
    ]
}

config_str = toml.dumps(config)

dataset_config = os.path.join(config_dir, "dataset_config.toml")

for key in config:
    if isinstance(config[key], dict):
        for sub_key in config[key]:
            if config[key][sub_key] == "":
                config[key][sub_key] = None
    elif config[key] == "":
        config[key] = None

config_str = toml.dumps(config)

with open(dataset_config, "w") as f:
    f.write(config_str)

print(config_str)

#@markdown ##<br>🔸 3.3 Cài đặt ảnh mẫu :
enable_sample = True
sample_every_n_type = "sample_every_n_epochs"
#@markdown <br>*Lưu mẫu thử sau mỗi n vòng lấy mẫu*
sample_every_n_type_value = 2 #@param {type:"number"}
if not enable_sample:
  sample_every_n_type_value = 999999
sampler = "euler_a"
#@markdown *Prompt mô tả ảnh mẫu*
prompt = "1girl in the beach, masterpiece,ultra realistic,32k,extremely detailed CG unity 8k wallpaper, best quality" #@param {type: "string"}
negative = "watermark,text, logo,contact, error, blurry, cropped, username, artist name, (worst quality, low quality:1.4),"
width = 512
height = 768 #@param {type:"number"}
scale = 7 
seed = -1 
steps = 20 

sample_str = f"""
  {prompt} \
  --n {negative} \
  --w {width} \
  --h {height} \
  --l {scale} \
  --s {steps} \
  {f"--d " + seed if seed > 0 else ""} \
"""

prompt_path = os.path.join(config_dir, "sample_prompt.txt")

with open(prompt_path, "w") as f:
    f.write(sample_str)

#@markdown ###<br>🔸 3.4 Setting Dim - Alpha ( 16 - 128)
#@markdown <br>
network_category = "LoRA"  # @param ["LoRA", "LoCon", "LoCon_Lycoris", "LoHa"]

# @markdown Cài đặt chỉ số tương ứng:

# @markdown | network_category | conv_dim | conv_alpha |network_dim | network_alpha | 
# @markdown | :---: | :---: | :---: | :---: | :---: |
# @markdown | LoRA | - | - | 32 | 1 |
# @markdown | LoCon | 8 | 1 | 16 | 8 |
# @markdown | LoHa | 4 | 1 | 8 | 4 |
# @markdown `conv_dim` và `conv_alpha` cần có khi đào tạo `LoCon` and `LoHa`, bỏ qua nó khi đào tạo là `LoRA`. Tuy nhiên, nếu lỗi nên đặt `dim = alpha`.
conv_dim = 1  # @param {'type':'number'}
conv_alpha = 1  # @param {'type':'number'}

network_dim = 32 #@param {'type':'number'}
network_alpha = 32 #@param {'type':'number'}
network_weight = ""
network_module = "lycoris.kohya" if network_category in ["LoHa", "LoCon_Lycoris"] else "networks.lora"
network_args = "" if network_category == "LoRA" else [
    f"conv_dim={conv_dim}", f"conv_alpha={conv_alpha}",
    ]

if network_category == "LoHa":
  network_args.append("algo=loha")
elif network_category == "LoCon_Lycoris":
  network_args.append("algo=lora")
#@markdown ### <br>🔹 3.5 Optimizer Config:
#@markdown <br>
optimizer_type = "AdamW8bit" #@param ["AdamW", "AdamW8bit", "Lion", "SGDNesterov", "SGDNesterov8bit", "DAdaptation", "AdaFactor"]
optimizer_args = "" 
train_unet = True
#@markdown ### <br>🔸 3.6 Learning rate:
#@markdown <br>*Nên đặt unet_lr = 2 x text_encoder_lr*
unet_lr = 1e-4 #@param {'type':'number'}
train_text_encoder = True
text_encoder_lr = 5e-5 #@param {'type':'number'}
lr_scheduler = "constant" #@param ["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup", "adafactor"] {allow-input: false}
lr_warmup_steps = 0 
lr_scheduler_num_cycles = 0 
lr_scheduler_power = 0

print("- LoRA Config:")
print(f"  - Additional network category: {network_category}")
print(f"  - Loading network module: {network_module}")
if not network_category == "LoRA":
  print(f"  - network args: {network_args}")
print(f"  - {network_module} linear_dim set to: {network_dim}")
print(f"  - {network_module} linear_alpha set to: {network_alpha}")
if not network_category == "LoRA":
  print(f"  - {network_module} conv_dim set to: {conv_dim}")
  print(f"  - {network_module} conv_alpha set to: {conv_alpha}")

if not network_weight:
    print("  - No LoRA weight loaded.")
else:
    if os.path.exists(network_weight):
        print(f"  - Loading LoRA weight: {network_weight}")
    else:
        print(f"  - {network_weight} does not exist.")
        network_weight = ""

print("- Optimizer Config:")
print(f"  - Using {optimizer_type} as Optimizer")
if optimizer_args:
    print(f"  - Optimizer Args: {optimizer_args}")
if train_unet and train_text_encoder:
    print("  - Train UNet and Text Encoder")
    print(f"    - UNet learning rate: {unet_lr}")
    print(f"    - Text encoder learning rate: {text_encoder_lr}")
if train_unet and not train_text_encoder:
    print("  - Train UNet only")
    print(f"    - UNet learning rate: {unet_lr}")
if train_text_encoder and not train_unet:
    print("  - Train Text Encoder only")
    print(f"    - Text encoder learning rate: {text_encoder_lr}")
print(f"  - Learning rate warmup steps: {lr_warmup_steps}")
print(f"  - Learning rate Scheduler: {lr_scheduler}")
if lr_scheduler == "cosine_with_restarts":
    print(f"  - lr_scheduler_num_cycles: {lr_scheduler_num_cycles}")
elif lr_scheduler == "polynomial":
    print(f"  - lr_scheduler_power: {lr_scheduler_power}")

#@markdown ##<br>🔻 3.7 Training config

import toml
import os
%store -r

lowram = True 
noise_offset = 0.0
#@markdown <br>*Cài đặt số vòng lấy mẫu*

num_epochs = 12 #@param {type:"number"}

train_batch_size = Batch_size
mixed_precision = "fp16" 
save_precision = "fp16" 
save_n_epochs_type = "save_every_n_epochs"
#@markdown <br>*Cài đặt sao lưu kết quả sau mỗi n vòng lẫy mẫu:*

save_n_epochs_type_value = 2 #@param {type:"number"}

save_model_as = "safetensors" 
max_token_length = 225
clip_skip = 1
gradient_checkpointing = False 
gradient_accumulation_steps = 1 
seed = -1
logging_dir = "/content/LoRA/logs"
prior_loss_weight = 1.0
              
os.chdir(repo_dir)

config = {
    "model_arguments": {
        "v2": v2,
        "v_parameterization": v_parameterization if v2 and v_parameterization else False,
        "pretrained_model_name_or_path": pretrained_model_name_or_path,
        "vae": vae,
    },
    "additional_network_arguments": {
        "no_metadata": False,
        "unet_lr": float(unet_lr) if train_unet else None,
        "text_encoder_lr": float(text_encoder_lr) if train_text_encoder else None,
        "network_weights": network_weight,
        "network_module": network_module,
        "network_dim": network_dim,
        "network_alpha": network_alpha,
        "network_args": network_args,
        "network_train_unet_only": True if train_unet and not train_text_encoder else False,
        "network_train_text_encoder_only": True if train_text_encoder and not train_unet else False,
        "training_comment": None,
    },
    "optimizer_arguments": {
        "optimizer_type": optimizer_type,
        "learning_rate": unet_lr,
        "max_grad_norm": 1.0,
        "optimizer_args": eval(optimizer_args) if optimizer_args else None,
        "lr_scheduler": lr_scheduler,
        "lr_warmup_steps": lr_warmup_steps,
        "lr_scheduler_num_cycles": lr_scheduler_num_cycles if lr_scheduler == "cosine_with_restarts" else None,
        "lr_scheduler_power": lr_scheduler_power if lr_scheduler == "polynomial" else None,
    },
    "dataset_arguments": {
        "cache_latents": True,
        "debug_dataset": False,
    },
    "training_arguments": {
        "output_dir": output_dir,
        "output_name": project_name,
        "save_precision": save_precision,
        "save_every_n_epochs": save_n_epochs_type_value if save_n_epochs_type == "save_every_n_epochs" else None,
        "save_n_epoch_ratio": save_n_epochs_type_value if save_n_epochs_type == "save_n_epoch_ratio" else None,
        "save_last_n_epochs": None,
        "save_state": None,
        "save_last_n_epochs_state": None,
        "resume": None,
        "train_batch_size": train_batch_size,
        "max_token_length": 225,
        "mem_eff_attn": False,
        "xformers": True,
        "max_train_epochs": num_epochs,
        "max_data_loader_n_workers": 8,
        "persistent_data_loader_workers": True,
        "seed": seed if seed > 0 else None,
        "gradient_checkpointing": gradient_checkpointing,
        "gradient_accumulation_steps": gradient_accumulation_steps,
        "mixed_precision": mixed_precision,
        "clip_skip": clip_skip if not v2 else None,
        "logging_dir": logging_dir,
        "log_prefix": project_name,
        "noise_offset": noise_offset if noise_offset > 0 else None,
        "lowram": lowram,
    },
    "sample_prompt_arguments":{
        "sample_every_n_steps": sample_every_n_type_value if sample_every_n_type == "sample_every_n_steps" else None,
        "sample_every_n_epochs": sample_every_n_type_value if sample_every_n_type == "sample_every_n_epochs" else None,
        "sample_sampler": sampler,
    },
    "dreambooth_arguments":{
        "prior_loss_weight": 1.0,
    },
    "saving_arguments":{
        "save_model_as": save_model_as
    },
}

config_path = os.path.join(config_dir, "config_file.toml")

for key in config:
    if isinstance(config[key], dict):
        for sub_key in config[key]:
            if config[key][sub_key] == "":
                config[key][sub_key] = None
    elif config[key] == "":
        config[key] = None

config_str = toml.dumps(config)

with open(config_path, "w") as f:
    f.write(config_str)

print(config_str)

# ⌛️ 4. Run

In [ ]:
#@markdown #▶️ Theo dõi quá trình và kiểm soát chất lượng
#@markdown *Kiểm tra sample tại ```{Output_Path}/sample```*
#@markdown <br>*Có thể dừng bất cứ khi nào sample thấy ưng ý*

os.chdir(repo_dir)
!accelerate launch \
  --config_file={accelerate_config} \
  --num_cpu_threads_per_process=1 \
  train_network.py \
  --sample_prompts=/content/LoRA/config/sample_prompt.txt \
  --dataset_config=/content/LoRA/config/dataset_config.toml \
  --config_file=/content/LoRA/config/config_file.toml

